In [1]:
import cv2
import numpy as np
import glob
from scipy import signal

In [2]:
def make_video(img_array, video_name):
    height, width, layers = img_array[0].shape
    size = (width,height)
    out = cv2.VideoWriter(video_name+'.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
    for i in range(len(img_array)):
        out.write(img_array[i])
    out.release()

In [3]:
def ssd(image, template):
    reserve = image.copy()
    h, w, _ = image.shape
    k_h, k_w, _ = template.shape
    ssd_scores = np.zeros((h - k_h, w - k_w))
    image = np.array(image, dtype="float")
    template = np.array(template, dtype="float")
    for i in range(0, h - k_h):
        for j in range(0, w - k_w):
            score = (image[i:i + k_h, j:j + k_w] - template)**2
            ssd_scores[i, j] = score.sum()
    min_points = np.unravel_index(ssd_scores.argmin(), ssd_scores.shape)
    return cv2.rectangle(reserve, (min_points[1], min_points[0]), (min_points[1]+k_w, min_points[0]+k_h), (0,0,255), 2)

In [4]:
def ncc(image, template):
    reserve = image.copy()    
    h, w, _ = image.shape
    k_h, k_w, _ = template.shape    
    ncc_scores = np.zeros((h - k_h, w - k_w))    
    image = np.array(image, dtype="float")
    template = np.array(template, dtype="float")
    
    for i in range(0, h - k_h):
        for j in range(0, w - k_w):
            image_sub = image[i:i + k_h, j:j + k_w]
            numerator = np.sum(image_sub * template)
            denominator = np.sqrt( (np.sum(image_sub ** 2))) * np.sqrt(np.sum(template ** 2))
            if(denominator == 0):
                ncc_scores[i, j] = 0
            else:
                ncc_scores[i, j] = numerator / denominator
                
    min_points = np.unravel_index(ncc_scores.argmax(), ncc_scores.shape)
    return cv2.rectangle(reserve, (min_points[1], min_points[0]), (min_points[1]+k_w, min_points[0]+k_h), (0,0,255), 2)

In [5]:
def cc(image, template):
    reserve = image.copy()

    t_height, t_width, _ = template.shape

    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    template = cv2.cvtColor(template, cv2.COLOR_RGB2GRAY)

    image = image.astype(np.float64)
    template = template.astype(np.float64)

    scores = signal.correlate2d(image, template, 'same')
    i, j = np.unravel_index(scores.argmax(), scores.shape)
    
    return cv2.rectangle(reserve, (i, j), (i + t_height, j + t_width), (0,0,255), 2)

In [6]:
temp = cv2.imread('template.jpg')
ncc_arr = []
ssd_arr = []
cc_arr = []
for filename in glob.glob('image_girl/*.jpg'):
    img_1 = cv2.imread(filename)
    ssd_arr.append(ssd(img_1, temp))
    cc_arr.append(cc(img_1, temp))
    ncc_arr.append(ncc(img_1, temp))

In [7]:
make_video(ncc_arr, "ncc")
make_video(ssd_arr, "ssd")
make_video(cc_arr, "cc")

In [8]:
cv2.imwrite( "ssd.jpg",ssd_arr[0])
cv2.imwrite( "cc.jpg",cc_arr[0])
cv2.imwrite( "ncc.jpg",ncc_arr[0])

True